## FP-Growth Frequent Pattern Mining

In [37]:
# Importing required libraries
import pandas as pd
import numpy as np
from itertools import combinations
import collections
import operator
import time

In [38]:
# Reading data file
adult = pd.read_excel('adult.xlsx')
adult.head()

,Age,Workclass,fnlwgt,Education,Edunum,Marital,Occupation,Relationship,Race,Sex,Gain,Loss,Hoursperweek,Country,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [39]:
# Removing irrelevant columns
adult.drop(['fnlwgt', 'Edunum'], axis = 1, inplace = True)
adult.head()

,Age,Workclass,Education,Marital,Occupation,Relationship,Race,Sex,Gain,Loss,Hoursperweek,Country,Income
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [40]:
# Size of original data
adult.shape

(32561, 13)

In [41]:
# Removing missing values from data
adult.replace(' ?', np.NaN, inplace = True)
adult.dropna(axis = 0, inplace = True)
adult.head()

,Age,Workclass,Education,Marital,Occupation,Relationship,Race,Sex,Gain,Loss,Hoursperweek,Country,Income
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [42]:
# Size of data after removing missing values and resetting index
adult.shape
adult.reset_index(drop = True, inplace = True)
adult.head()

,Age,Workclass,Education,Marital,Occupation,Relationship,Race,Sex,Gain,Loss,Hoursperweek,Country,Income
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [43]:
# Converting numeric data to categorical data
adult['Age'] = pd.cut(adult['Age'], [0, 25, 40, np.inf], labels=["young", "middle_aged", "old"])
adult['Gain'] = pd.cut(adult['Gain'], [-1, 1, np.inf], labels=["No_Gain", "Gain"])
adult['Loss'] = pd.cut(adult['Loss'], [-1, 1, np.inf], labels=["No_Loss", "Loss"])
adult['Hoursperweek'] = pd.cut(adult['Hoursperweek'], [-1, 5, 20, 60, np.inf], labels=["Less", "Medium", "Reasonable", "High"])
adult.head()

,Age,Workclass,Education,Marital,Occupation,Relationship,Race,Sex,Gain,Loss,Hoursperweek,Country,Income
0,middle_aged,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,Gain,No_Loss,Reasonable,United-States,<=50K
1,old,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,No_Gain,No_Loss,Medium,United-States,<=50K
2,middle_aged,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,No_Gain,No_Loss,Reasonable,United-States,<=50K
3,old,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,No_Gain,No_Loss,Reasonable,United-States,<=50K
4,middle_aged,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,No_Gain,No_Loss,Reasonable,Cuba,<=50K


In [44]:
# Generating frequent 1 itemsets and their support count
def frequent(data):
    item = {} # empty dictionary to store each item as key and its count as value
    for i in data.index:
        for j in set(data.loc[i]):
            if j in item: 
                item[j] += 1
            else: 
                item[j] = 1
    return item

In [45]:
# Determining all the items satisfying minimum support threshold and sorting them
def items_above_threshold(item, support):
    freqItems = {}
    freqItems = dict((k,v) for k, v in item.items() if v >= support)
    
    # Sorting all the items
    freqItems = dict(sorted(freqItems.items(), reverse = True, key=operator.itemgetter(1)))
    
    print("\nAll frequent items along with their support count: \n", freqItems)
    return freqItems

In [46]:
# Generating ordered itemsets
def ordered_items(freqItems, data):
    # An empty list of lists to store all the ordered itemsets
    item = [[]for i in range(len(data))]
    
    for k,v in freqItems.items():
        for i in data.index:
            for j in data.loc[i]:
                if (k == j) and k not in item[i]:
                    item[i].append(k)
    
    return item

In [47]:
# Class for nodes of FP-Tree
class fp_tree(object):
    def __init__(self, n, c, p):
        self.name = n # Name of the node
        self.count = c # Support Count of the node item
        self.parent = p # Parent of the node
        self.children = {} # Stores all the children of the node
        
    # Function to increment count of a node by 1
    def increment(self):
        self.count += 1
        
    # Function to display FP Tree
    def display(self, i = 1):
        print ('   |'*i, self.name, '', self.count)
        for child in self.children.values():
            child.display(i+1)

In [48]:
# Function to create FP-Tree and return updated header
def tree(orderedItems, rootNode, header):
    for itemset in orderedItems:
        insert_node(itemset, rootNode, header)
    return header

In [49]:
# Function to insert a node in the tree and update header
def insert_node(itemset, rootNode, header):
    
    for i in itemset:
            if i in rootNode.children:
                rootNode.children[i].increment() # if node already present, increase its count by 1
            else:
                newNode = fp_tree(i, 1, rootNode)
                rootNode.children[i] = newNode # if node is not present already, add a new node
                header[i].append(newNode)
            
            rootNode = rootNode.children[i]
    
    return header

In [50]:
# Function to generate conditional pattern base
def conditional_pb(header):
    patterns = {} # Stores conditional pattern base
    
    for i in header:
        patterns[i] = [] # Create keys for all the items in header
        
    for i in header:
        for node in header[i]:
            pat = [] # To store name and count of the node traversed
            while (node.parent != None): # Travel through the tree till root node is reached
                # Add all the nodes except for the node for which pattern base is being created
                # Add count of the node for which pattern base is being created
                if node.name != i:
                    pat.append(node.name)
                else:
                    pat.append(node.count)
                node = node.parent
                
            patterns[i].append(pat)
    
    # Rearranging patterns and corresponding counts
    for i in patterns:
        temp = []
        for value in patterns[i]:
            count = value[0]
            pat = [value[j] for j in range(1, len(value))] 
            temp.append((pat, count))
        patterns[i] = temp 
    
    return patterns

In [51]:
# Function to generate conditional FP Tree
def conditional_tree(patternBase):

    condnlFpTree = {} # Dictionary tp store conditional FP Tree
    for i in patternBase:
        condnlFpTree[i] = tuple()

    for i in patternBase:
        if (len(patternBase[i]) > 1):
            count = patternBase[i][0][1]
            temp = patternBase[i][0][0]
            for j in range(1, len(patternBase[i])):
                count += patternBase[i][j][1]
                temp = list(set(temp).intersection(set(patternBase[i][j][0])))
            condnlFpTree[i] = temp, count
        else:
            condnlFpTree[i] = patternBase[i][0]

    return condnlFpTree

In [52]:
# Function to generate all the frequent patterns
def freq_patterns(condnlFpTree):
    fPats = {} # Dictionary to store all the frequent patterns
    
    for i in condnlFpTree:
        if condnlFpTree[i][0] != []:
            c = len(condnlFpTree[i][0])
            for k in range(len(condnlFpTree[i][0])):
                x = set((i , condnlFpTree[i][0][k]))
                fPats[tuple(x)] = condnlFpTree[i][1]
                x = set((i, condnlFpTree[i][0][k], condnlFpTree[i][0][k-1]))
                fPats[tuple(x)] = condnlFpTree[i][1]
            
    return fPats

In [67]:
# Function to implement FP Growth Algorithm
def fp_growth(data, support):
    
    t1 = time.time()
    s = int((support/100)*len(data))
    print("Minimum support threshold = ", s)
    
    items = {}
    items = frequent(data)
    
    freqItems = {}
    freqItems = items_above_threshold(items, s)
    
    oI = ordered_items(freqItems, data)
    
    header = {}
    for i in freqItems:
        header[i] = []
    
    rootNode = fp_tree('root', 1, None)
    header = tree(oI, rootNode, header)
    print("\n")
    rootNode.display()
    
    condPat = conditional_pb(header)
    print("\nConditional Pattern Base:\n", condPat)
    
    condFpTree = conditional_tree(condPat)
    print("\nConditional FP Tree:\n", condFpTree)
    
    fPatterns = freq_patterns(condFpTree)
    print("\nFrequent Patterns Generated using FP Growth Algorithm:\n", fPatterns)
    
    t2 = time.time()
    
    execTime = t2 - t1
    
    print("\nExecution Time: ", execTime)

    return

In [68]:
fp_growth(adult, 0.5)

Minimum support threshold =  150

All frequent items along with their support count: 
 {'No_Loss': 28735, 'No_Gain': 27624, ' United-States': 27504, 'Reasonable': 26722, ' White': 25933, ' <=50K': 22654, ' Private': 22286, ' Male': 20380, ' Married-civ-spouse': 14065, ' Husband': 12463, 'old': 12402, 'middle_aged': 12092, ' HS-grad': 9840, ' Female': 9782, ' Never-married': 9726, ' Not-in-family': 7726, ' >50K': 7508, ' Some-college': 6678, 'young': 5668, ' Bachelors': 5044, ' Own-child': 4466, ' Divorced': 4214, ' Prof-specialty': 4038, ' Craft-repair': 4030, ' Exec-managerial': 3992, ' Adm-clerical': 3721, ' Sales': 3584, ' Other-service': 3212, ' Unmarried': 3212, ' Black': 2817, 'Gain': 2538, ' Self-emp-not-inc': 2499, 'Medium': 2277, ' Local-gov': 2067, ' Machine-op-inspct': 1966, ' Masters': 1627, ' Transport-moving': 1572, 'Loss': 1427, ' Wife': 1406, ' Handlers-cleaners': 1350, ' Assoc-voc': 1307, ' State-gov': 1279, ' Self-emp-inc': 1074, 'High': 1052, ' 11th': 1048, ' Assoc-a